# Advanced RAG on Hugging Face Collections with LangChain, Cohere, and Llama 3

Authored by: [Justin Goheen](https://huggingface.co/jxtngx)

This demo shows how to perform advanced Retrieval Augmented Generation (RAG) on documents contained in a Hugging Face Collection.

For an introduction to advanced RAG, you can check out this [other cookbook](https://huggingface.co/learn/cookbook/en/advanced_rag), by [Aymeric Roucher](https://huggingface.co/m-ric).

We will use the following tools:

<div>
    <table>
        <tr>
            <th>requirement</th>
            <th>purpose</th>
            <th>link</th>
        </tr>
        <tr>
            <td>LangChain</td>
            <td>LLM workflow framework</td>
            <td><a href="https://python.langchain.com/v0.2/docs/introduction/">docs</a></td>
        </tr>
        <tr>
            <td>Comet LLM</td>
            <td>tracking LLM workflows</td>
            <td><a href="https://www.comet.com/docs/v2/guides/comet-llm/quickstart/">docs</a></td> 
        </tr>
        <tr>
            <td>Unstructured</td>
            <td>document processing</td>
            <td><a href="https://docs.unstructured.io/welcome">docs</a></td> 
        </tr>
        <tr>
            <td>Llama 3 70B Instruct</td>
            <td>synthesizer model</td>
            <td><a href="https://huggingface.co/meta-llama/Meta-Llama-3-70B-Instruct">docs</a></td> 
        </tr>
        <tr>
            <td>Cohere Reranker</td>
            <td>reranking model</td>
            <td><a href="https://cohere.com/rerank">docs</a></td> 
        </tr>
        <tr>
            <td>Hugging Face Hub API</td>
            <td>interact with the HF Hub</td>
            <td><a href="https://huggingface.co/docs/huggingface_hub/index">docs</a></td> 
        </tr>
        <tr>
            <td>Hugging Face Inference API</td>
            <td>serverless inference for prototyping</td>
            <td><a href="https://huggingface.co/docs/api-inference/index">docs</a></td> 
        </tr>
        <tr>
            <td>Weaviate</td>
            <td>vector database</td>
            <td><a href="https://huggingface.co/docs/api-inference/index">docs</a></td> 
        </tr>
        <tr>
            <td>python-dotenv</td>
            <td>reading environment variables</td>
            <td><a href="https://saurabh-kumar.com/python-dotenv/">docs</a></td> 
        </tr>
    </table>
</div>

## Installs

<ol>
    <li>langchain</li>
    <li>langchain-weaviate</li>
    <li>langchain-cohere</li>
    <li>langchain-huggingface</li>
    <li>langchain-community</li>
    <li>weaviate-client</li>
    <li>comet-llm</li>
    <li>huggingface-hub</li>
    <li>unstructured[all-docs]</li>
</ol>

In [1]:
!pip -q install langchain huggingface-hub comet-llm "unstructured[all-docs]" weaviate-client langchain-weaviate langchain-cohere langchain-huggingface langchain-community python-dotenv

## Steps

We need to do the following:

<ol>
  <li>create keys for: <a href="https://www.comet.com/site/">Comet</a>, <a href="https://unstructured.io/api-key-free">Unstructured</a>, <a href="https://weaviate.io/">Weaviate</a>, and <a href="https://cohere.com/">Cohere</a></li>
  <li>request access to the Llama 3 models on the Hugging Face Hub</li>
  <li>create an access token on <a href="https://huggingface.co/docs/hub/en/security-tokens#user-access-tokens">Hugging Face Hub</a></li>
  <li>Create the <a href="https://huggingface.co/docs/hub/collections">collection</a> on Hugging Face</li>
  <li>Fetch the collection with the Hugging Face Hub API</li>
  <li>Preprocess the documents contained in the collection with Unstructured</li>
  <li>Use Weaviate and LangChain to create a vector store and retriever</li>
  <li>Use Hugging Face's Inference API to synthesize an answer using Meta's Llama 3 70B Instruct</li>
</ol>

# Breaking down RAG

Lorem ipsum dolor sit amet

## Retrieval

Lorem ipsum dolor sit amet

### Vector search with Hierarchical Navigable Small Worlds (HNSW)

Lorem ipsum dolor sit amet

### Embedding models

Lorem ipsum dolor sit amet

### Reranking models

Lorem ipsum dolor sit amet

## Synthesis aka Generation

Lorem ipsum dolor sit amet

# Combining LangChain and Comet LLM

See the Comet LLM [docs](https://www.comet.com/docs/v2/guides/comet-llm/integrations/langchain/) for more.

Lorem ipsum dolor sit amet

In [2]:
import os
import comet_llm

from langchain_community.callbacks.tracers.comet import CometTracer

In [3]:
comet_llm.init(project="hf-cookbook-advanced-rag", api_key=os.environ["COMET_API_KEY"])

COMET INFO: Valid Comet API Key saved in /Users/justin/.comet.config (set COMET_CONFIG to change where it is saved).


## What is LangChain

Lorem ipsum dolor sit amet

## What is Comet LLM

Lorem ipsum dolor sit amet

# The Workflow

In [4]:

import logging
from textwrap import wrap

from dotenv import load_dotenv

load_dotenv()

True

## Get the collection and download the files

In [5]:
from huggingface_hub import get_collection

/Users/justin/Developer/huggingface/hf-cookbook/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ensure the data directory exists

In [6]:
docdir = os.path.join(".", "documents")

if not os.path.isdir(docdir):
    os.mkdir(docdir)

get the collection's files

In [7]:
# get data dir contents
data = os.listdir(docdir)
# get tool use paper collection
collection = get_collection("jxtngx/tool-use-papers-664c6cd9cc9c64354af51e86")
# make arxiv urls
urls = ["".join(["https://arxiv.org/pdf/", c.item_id]) for c in collection.items]

# download files
for url in urls:
    docname = "".join([url.split('/')[-1], ".pdf"])
    if not os.path.exists(os.path.join(docdir, docname)):
        os.system(f"wget -O {os.path.join(docdir, docname)} {url}")

## Prep PDFs with Unstructured

This section takes inspiration from the example in [Building RAG with Custom Unstructured Data](https://github.com/huggingface/cookbook/blob/main/notebooks/en/rag_with_unstructured_data.ipynb), by Maria Khalusova.

See the sections titled `Unstructured data preprocessing` and `Chunking` of that example for a detailed walk through provided by Maria.

In [8]:
from unstructured.ingest.connector.local import SimpleLocalConfig
from unstructured.ingest.interfaces import PartitionConfig, ProcessorConfig, ReadConfig
from unstructured.ingest.runner import LocalRunner
from unstructured.staging.base import elements_from_json
from unstructured.chunking.title import chunk_by_title
from langchain_core.documents import Document

In [9]:
# Optional cell to reduce the amount of logs
logger = logging.getLogger("unstructured.ingest")

if logger.root.handlers:
    logger.root.removeHandler(logger.root.handlers[0])

In [10]:
output_path = "./local-ingest-output"

runner = LocalRunner(
    processor_config=ProcessorConfig(
        # logs verbosity
        verbose=True,
        # the local directory to store outputs
        output_dir=output_path,
        num_processes=2,
        ),
    read_config=ReadConfig(),
    partition_config=PartitionConfig(
        partition_by_api=True,
        api_key=os.environ["UNSTRUCTURED_API_KEY"],
        ),
    connector_config=SimpleLocalConfig(
        input_path="./documents",
        # whether to get the documents recursively from given directory
        recursive=False,
        ),
    )
runner.run()

2024-06-26 19:24:32,586 MainProcess INFO     running pipeline: DocFactory -> Reader -> Partitioner -> Copier with config: {"reprocess": false, "verbose": true, "work_dir": "/Users/justin/.cache/unstructured/ingest/pipeline", "output_dir": "./local-ingest-output", "num_processes": 2, "raise_on_error": false}
2024-06-26 19:24:32,680 MainProcess INFO     Running doc factory to generate ingest docs. Source connector: {"processor_config": {"reprocess": false, "verbose": true, "work_dir": "/Users/justin/.cache/unstructured/ingest/pipeline", "output_dir": "./local-ingest-output", "num_processes": 2, "raise_on_error": false}, "read_config": {"download_dir": "", "re_download": false, "preserve_downloads": false, "download_only": false, "max_docs": null}, "connector_config": {"input_path": "./documents", "recursive": false, "file_glob": null}}
2024-06-26 19:24:32,684 MainProcess INFO     processing 2 docs via 2 processes
2024-06-26 19:24:32,685 MainProcess INFO     Calling Reader with 2 docs
202

In [11]:
elements = []

for filename in os.listdir(output_path):
    filepath = os.path.join(output_path, filename)
    elements.extend(elements_from_json(filepath))

In [12]:
chunked_elements = chunk_by_title(elements,
                                  # maximum for chunk size
                                  max_characters=512,
                                  # You can choose to combine consecutive elements that are too small
                                  # e.g. individual list items
                                  combine_text_under_n_chars=200,
                                  )

In [13]:
documents = []

for chunked_element in chunked_elements:
    metadata = chunked_element.metadata.to_dict()
    metadata["source"] = metadata["filename"]
    del metadata["languages"]
    documents.append(Document(page_content=chunked_element.text, metadata=metadata))

## Create the vector store and retriever

See the [LangChain docs](https://python.langchain.com/v0.2/docs/integrations/text_embedding/huggingfacehub/) for more on the Hugging Face embeddings integration.

See the [LangChain docs](https://python.langchain.com/v0.1/docs/integrations/vectorstores/weaviate/) for more on the Weaviate integration.

In [14]:
import weaviate
from weaviate.auth import AuthApiKey

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.chains import RetrievalQA

from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_cohere import CohereRerank

Connect to Weaviate Cloud

In [15]:
weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
)

In [16]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.environ["HF_INFERENCE_API_KEY"], 
    model_name="BAAI/bge-base-en-v1.5"
)
vectorstore = WeaviateVectorStore.from_documents(documents, embeddings, client=weaviate_client)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [17]:
compressor = CohereRerank(cohere_api_key=os.environ["COHERE_API_KEY"])
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

## Using Hugging Face Inference Endpoints for prototyping

See the [LangChain docs](https://python.langchain.com/v0.2/docs/integrations/llms/huggingface_endpoint/) for more.

In [18]:
from langchain_huggingface import HuggingFaceEndpoint

In [19]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-70B-Instruct",
    model_kwargs={"max_length": 128},
    temperature=0.5,
    huggingfacehub_api_token=os.environ["HF_INFERENCE_API_KEY"],
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/justin/.cache/huggingface/token
Login successful


In [20]:
chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=compression_retriever,
)

In [21]:
result = chain.invoke("What is chain of abstraction", config = {'callbacks' : [CometTracer()]})['result']

Chain logged to https://www.comet.com/jxtngx/hf-cookbook-advanced-rag


In [22]:
for sequence in wrap(result):
    for token in sequence.split():
        print(token, end=" ", flush=True)
    print()

Chain of abstraction (CoA) is a method that decouples general 
reasoning from domain-specific knowledge. It involves fine-tuning 
large language models (LLMs) to generate abstract reasoning chains 
with placeholders, which are then replaced with domain-specific 
knowledge obtained from external tools. 
